# Build graph from patients similarity
In this notebook, we test some methods to build the graph from an initial dataframe.

In [10]:
import pandas as pd
import numpy as np
from models.BuildGraph import BuildGraph
from utils.preProcessing import preProcessing

In [11]:
preprocessor = preProcessing()

## 1. Reading dataset

In [12]:
df = pd.read_excel('data/Supplemental-table1.xlsx')
df = df.loc[df["Cohort"] == "Naive"]
df

,Anonymous ID,EGA ID,Tumour type,Sex,Line of therapy,Immunotherapy regimen,Cohort,Reason for discontinuation,Best response,Age at advanced disease diagnosis,...,Alive_0,Time to progression (days),Progression_1,Clinical benefit,CD8+ T cell score,Exome mut per mb,Genome mut per mb,CD274 expression,M1M2 expression,Lymph related
0,14891,EGAD00001001961,LUNG,F,8,Nivolumab,Naive,Progression,Mixed,45,...,1,179,1,NCB,0.351869,11.095310,23.0729,4.1689,55.51575,Non-lymph related
1,18624,EGAD00001002047,AECA,F,10,Nivolumab,Naive,Progression,Stable disease,47,...,1,148,1,NCB,0.071464,3.876336,5.4552,0.7910,9.32352,Non-lymph related
2,21392,EGAD00001002544,OV,F,6,Monalizumab,Naive,Progression,Physician assessed SD,57,...,1,79,1,NCB,0.040394,2.415688,4.1274,4.0377,50.12008,Lymph related
3,21516,EGAD00001002546,SKCM,F,3,Ipilimumab,Naive,Death,Not evaluable,29,...,1,11,0,NCB,0.052774,6.235845,11.4871,0.6006,1.58550,Non-lymph related
4,22186,EGAD00001001966,LUNG,F,4,Atezolizumab,Naive,Progression,Stable disease,52,...,1,80,1,NCB,0.009323,4.606660,7.1503,1.8234,38.42695,Non-lymph related
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,39280,EGAD00001004933,LUNG,M,2,Pembrolizumab,Naive,Toxicity,Physician Assessed SD,60,...,1,401,0,DCB,0.695396,84.998501,96.4079,8.4768,86.62561,Non-lymph related
94,37899,EGAD00001005838,CHOL,F,2,Nivolumab,Naive,Death,Physician assessed PR,38,...,1,96,0,NCB,0.818233,2.247151,2.7416,11.2704,47.97376,Non-lymph related
95,37775,EGAD00001004934,HNSC,M,1,Avelumab/OX40 agonist PF-04518600,Naive,Progression,Stable disease,69,...,1,69,1,NCB,0.018982,3.848247,5.8036,8.7136,5.13403,Non-lymph related
96,38313,EGAD00001005844,ACC,F,3,Avelumab/Debio1143 (SMAC mimetic),Naive,Toxicity,Partial response,34,...,1,51,1,NCB,0.000000,5.617878,7.7940,0.0688,0.51364,Non-lymph related


## 2. Find nearest patients
The useful features to evaluate patients similarity are :
- Tumour type
- Sex
- Immunotherapy regimen
- Age
- CD8+ T cell score
- M1M2 expression score
- Genome mut per mb
- Exome mut per mb
- CD274 expression

Some of these features are categorical, so we need to numerize them.

### a. Numerize categorical features and normalization

In [13]:
df_numerized = pd.get_dummies(df.loc[:,["Tumour type","Sex","Immunotherapy regimen","Age at advanced disease diagnosis","CD8+ T cell score","M1M2 expression","Genome mut per mb","Exome mut per mb","CD274 expression"]]).astype(float)
X_numerized = preprocessor.normalize_data(df_numerized.to_numpy())

### b. Compute adjacency matrix

In [14]:
# 1 neighbor per patient, and euclidean distance
buildgraph = BuildGraph(n_neighbors=2, metric="euclidean")

In [15]:
A = buildgraph.build_adjacency_matrix(X_numerized)
A

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

## 3. Split per tumour type

In [21]:
tumour_types = np.unique(df["Tumour type"].to_numpy())
dico_tumour = {} #clé : tumour type, value = df_associated
for i in range(tumour_types.shape[0]):
    dico_tumour[tumour_types[i]] = df.loc[df["Tumour type"] == tumour_types[i]]

In [22]:
dico_tumour

{'ACC':     Anonymous ID           EGA ID Tumour type Sex Line of therapy   
 33         28123  EGAD00001003065         ACC   M               2  \
 96         38313  EGAD00001005844         ACC   F               3   
 
                 Immunotherapy regimen Cohort Reason for discontinuation   
 33           Tremelimumab, Durvalumab  Naive                Progression  \
 96  Avelumab/Debio1143 (SMAC mimetic)  Naive                   Toxicity   
 
           Best response  Age at advanced disease diagnosis  ...  Alive_0   
 33  Progressive disease                                 40  ...        0  \
 96     Partial response                                 34  ...        1   
 
     Time to progression (days)  Progression_1  Clinical benefit   
 33                         107              1               NCB  \
 96                          51              1               NCB   
 
    CD8+ T cell score  Exome mut per mb  Genome mut per mb  CD274 expression   
 33          0.068854          0